In [ ]:
import pathlib
import plotly.express as px
import pandas as pd
from difflib import diff_bytes
import numpy as np
from dash import Dash, dcc, html, Output, Input, dash_table  # pip install dash
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
from jupyter_dash import JupyterDash
from countryinfo import CountryInfo
import pycountry



In [ ]:

def add_alpha_codes_from_ISO(df, col):
    """
    adds a column of alpha3 codes to a dataframe with country name in column 'col'
    """
    input_countries = df[col]
    countries = []
    for input_country in input_countries:
        try:
            country = pycountry.countries.get(alpha_3=input_country)
            alpha2 = country.alpha_2
        except:
            alpha2 = "unk_" + str(input_country)
        countries.append(alpha2)
    df["alpha2"] = countries
    return df



In [ ]:

df_input = pd.read_csv("../data/country_data_master.csv")
# df_input = pd.read_csv("../data/country_data_master.csv", index_col="alpha3")


In [ ]:
iso_num_col = "ISO Code"
col = "Country"
alpha3_col = "alpha3"
alpha2_col = "alpha2"
df_original = df_input



df = add_alpha_codes_from_ISO(df_original, alpha3_col)

# filename = "scratch_new_country_data"
# df.to_csv(filename + ".csv")


Cell below appends the region, subregion, and bordering countries to the country. This is for future use for interpolation.

In [ ]:
# input_countries = df[alpha2_col]
# bordering_countries = []
# region = []
# subregion = []
# for input_country in input_countries:
#     try:
#         country = CountryInfo(input_country)
#     except:
#         print("couldn't find country")

#     try:
#         country_bordering_countries = ",".join(str(x) for x in country.borders())
#     except:
#         country_bordering_countries = "unk_" + str(input_country)
    
#     try:
#         country_region = country.region()
#     except:
#         country_region = "unk_" + str(input_country)

#     try:
#         country_subregion = country.subregion()
#     except:
#         country_subregion = "unk_" + str(input_country)
    


    
#     bordering_countries.append(country_bordering_countries)
#     region.append(country_region)
#     subregion.append(country_subregion)
# df["borders"] = bordering_countries
# df["region"] = region
# df["subregion"] = subregion




In [18]:
list_of_vars = ['PBO',
       'Terrain Ruggedness', 'Urban %', 'Urban Agg %', 'RoadQuality', 'Km',
       'Urb %', 'Average household size (number of members)',
       'National At Least Basic', 'National Limit (more than 30 mins)',
       'National Unimproved', 'National Surface Water',
       'Nat Accesible On Premises', 'Nat Piped', 'Nat NonPiped']


alpha2_col = "alpha2"


df_input = pd.read_csv("../data/Country Data before Interp.csv", index_col="alpha3")
df_output = df_input

df_interp_track = pd.DataFrame(df_input.index)
#set index to alpha3
df_interp_track = df_interp_track.set_index(alpha3_col)
df_interp_track[list_of_vars]="none"


#SO MANY ARE ISLANDS, WITH NO BORDERS!



for variable in list_of_vars:
    # convert column to float
    df_input[variable] = df_input[variable].astype(float)

    ## for loop in the future
    #find location of nan values
    nan_locations = df_input[variable].isna()
    countries_requiring_interp = df_input[nan_locations].index

    #next for loop goes here 
    for country in countries_requiring_interp:
        #see if the borders can be used to interpolate
        borders = df_input.loc[country, "borders"]
        borders = str(borders)
        border_list = borders.split(",")
        # see if the subregion can be used to interpolate
        subregion_list = df_input[df_input["subregion"]==df_input.loc[country, "subregion"]].index
        avg_var_subregion = (df_input.loc[subregion_list, variable].mean())
        # assume the region CAN be used to interpolate
        region_list = df_input[df_input["region"]==df_input.loc[country, "region"]].index
        avg_var_region = (df_input.loc[region_list, variable].mean())
        # try border, then subregion, then region
        if borders != "nan":
            avg_var_border = (df_input.loc[border_list, variable].mean())
            df_output.loc[country, variable] = avg_var_border
            df_interp_track.loc[country, variable] = "border interpolated"
        elif ~np.isnan(avg_var_subregion):
            df_output.loc[country, variable] = avg_var_subregion
            df_interp_track.loc[country, variable] = "subregion interpolated"
        else:
            df_output.loc[country, variable] = avg_var_region
            df_interp_track.loc[country, variable] = "region interpolated"



# filename = "scratch_new_country_data_before_interp"
# df.to_csv(filename + ".csv")

# #set index of pandas dataframe to alpha2
# # df = df.set_index(alpha2_col)
# country = CountryInfo(input_country)
# bordering_countries = country.borders()
# print(bordering_countries)

# #find the countries that border the country that needs interpolation
# bordering_countries_values = df.loc[bordering_countries]

# # df[input_country][bordering_countries]

# # borders = []
# # missing_countries = []
# # for input_country in countries_requiring_interp:
# #     try:
# #         country = CountryInfo(input_country)
# #         bordering_countries = country.borders()
# #         df[input_country][bordering_countries]


# #     except:
# #         missing_countries.append(input_country)
# # df["alpha2"] = countries



In [19]:
# df_input[variable].isna()


filename = "interpTrack2"
df_interp_track.to_csv(filename + ".csv")
filename = "interp2"
df_output.to_csv(filename + ".csv")
